## A/B-тест (приложение для онлайн-знакомств)

Вы работаете аналитиком в приложении для онлайн-знакомств. Механика приложения следующая: пользователи видят в приложении анкеты друг друга и могут ставить друг другу лайки или дизлайки. Если пользователи поставили друг другу лайк – это называется мэтч, и у пользователей появляется возможность познакомиться.

Команда приложения разработала новый алгоритм для поиска наиболее подходящих анкет. Для проверки работы алгоритма был проведен АБ-тест. Все пользователи были разделены на две группы. Пользователи в группе с номером 0 пользовались приложением со старым алгоритмом. Все пользователи в группе 1 пользовались приложением с новым алгоритмом для поиска анкет.

Ваша задача – оценить, правда ли, что новый алгоритм улучшил качество сервиса. Для этого нужно выбрать одну или несколько метрик, которые отвечают за качество сервиса, и статистически сравнить эти метрики в двух группах.

В данных находится выгрузка логов взаимодействия пользователей друг с другом. Для каждой пары пользователей указано, из какой они группы АБ-теста и случился ли у них мэтч.

Результат вашей работы – аналитическое заключение с ответом на вопрос, стоит ли включать новую систему поиска анкет на всех пользователей.

## 1. Предварительный анализ данных

In [1]:
# Импортируем библиотеки

import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency, chi2 

In [2]:
# Выгрузим данные

df = pd.read_csv('dating_data.csv')

## 1.1 Посмотрим на структуру данных, кол-во строк, тип данных, пропущенные значения

In [3]:
df.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [4]:
df.shape

(14514, 4)

In [5]:
df.dtypes

user_id_1    int64
user_id_2    int64
group        int64
is_match     int64
dtype: object

In [6]:
df.isna().sum()

user_id_1    0
user_id_2    0
group        0
is_match     0
dtype: int64

## 1.2 Проанализируем порядок формирования строк в датасете

In [7]:
# Кол-во строк в таблице больше, чем уникальных юзеров, т.е. 1 юзер встречается несколько раз

df.user_id_1.nunique()

1000

In [8]:
df.user_id_2.nunique()

1000

In [9]:
# Проверим, совпадают ли юзеры в столбцах user_id_1 и user_id_2. 
# Все юзеры из столбца user_id_1 есть в столбце user_id_2

df.user_id_1.isin(df.user_id_2).value_counts()

True    14514
Name: user_id_1, dtype: int64

In [10]:
# И наоборот, все юзеры из столбца user_id_2 есть в столбце user_id_1. То есть всего в датасете есть информация о 1000 юзеров

df.user_id_2.isin(df.user_id_1).value_counts()

True    14514
Name: user_id_2, dtype: int64

## 1.3 Посмотрим распределение юзеров в разбивке по группам (1 - новый алгоритм, 0 - старый алгоритм)

In [11]:
# 9722 взаимодействия произошло по новому алгоритму, 4792 - по старому

df.groupby('group', as_index = False) \
    .agg({'user_id_1' : 'count'})

,group,user_id_1
0,0,4792
1,1,9722


In [12]:
# количество уникальных юзеров распределено практически поровну: 499 - новый алгоритм, 501 - старый алгоритм

df.groupby('group', as_index = False) \
    .agg({'user_id_1' : 'nunique'})

,group,user_id_1
0,0,501
1,1,499


In [13]:
# Посмотрим, нет ли одних и тех же уникальных юзеров и в группе 0, и в группе 1
# Выбираем id юзеров в группе 0

users_0 = df.query('group == 0') \
            .groupby('user_id_1', as_index = False) \
            .agg({'user_id_2' : 'count'}) \
            .rename(columns = {'user_id_2' : 'interactions'})
            
users_0.head()

,user_id_1,interactions
0,4,9
1,10,6
2,12,12
3,14,8
4,15,20


In [14]:
# Выбираем id юзеров в группе 1

users_1 = df.query('group == 1') \
            .groupby('user_id_1', as_index = False) \
            .agg({'user_id_2' : 'count'}) \
            .rename(columns = {'user_id_2' : 'interactions'})
            
users_1.head()

,user_id_1,interactions
0,1,24
1,2,16
2,3,16
3,5,22
4,6,22


In [15]:
# Проверяем, есть ли id юзеров группы 0 в id юзеров в группе 1
# Пересекающихся юзеров в группе 0 и группе 1 нет

users_0.user_id_1.isin(users_1.user_id_1).sum()

0

## Делаем вывод
Всего в датасете содержится информация о 1000 юзеров (499 в группе 1, 501 в группе 0).  
1 строка в датасете - это одно взаимодействие 2 пользователей. Один и тот же юзер может встретиться в таблице несколько раз как в столбце user_id_1, так и в столбце user_id_2.   
Кол-во взаимодействий для разных юзеров разное. Общее кол-во взаимодействий в группе 1 в 2 раза больше, чем в группе 0 (примерно при одинаковом кол-ве юзеров).

## 2. Выбор метрики

Так как кол-во взаимодействий меняется от юзера к юзеру (сколько он просмотрит анкет) и распределение юзеров примерно
одинаково между 2 группами (499/501), я делаю допущение, что можно не учитывать число уникальных юзеров, а 
в качестве метрики использовать отношение мэтчей к общему числу взаимодействий (**% мэтчей**).

In [16]:
# Посчитаем данные для расчета метрики

match_tab = pd.crosstab(df.group, df.is_match)
match_tab

is_match,0,1
group,,
0,3858,934
1,5813,3909


In [17]:
# Посчитаем общее число взаимодействий и найдем отношение мэтчей к общему числу взаимодействий (% мэтчей)

match_tab['total'] = (match_tab[0] + match_tab[1])
match_tab['match'] = (match_tab[1] / match_tab.total).mul(100)
match_tab

is_match,0,1,total,match
group,,,,
0,3858,934,4792,19.490818
1,5813,3909,9722,40.207776


Как мы видим, % мэтчей в группе 1 (использование нового алгоритма) в 2 раза выше, чем в группе 0 (использование старого алгоритма). Проверим, являются ли данные результаты статистически значимыми.

## 3. Статистический тест

У нас есть 2 категориальные переменные: группа (0,1) и мэтч (0,1). Наша метрика похожа на конверсию. Поэтому статистическую значимость мы будем проверять с помощью критерия хи-квадрат.  

Зададим гипотезы:  
- $H_0$: взаимосвязи между переменными нет 
- $H_1$: взаимосвязь есть

In [18]:
stat, p, dof, expected = chi2_contingency(pd.crosstab(df.group, df.is_match))

In [19]:
stat, p

(618.6889141576198, 1.4418299163662586e-136)

In [20]:
if p < 0.05:
    print ('Отклоняем Ho')
else:
    print ('Отклоняем H1')

Отклоняем Ho


p-value меньше 0,05, значит мы отклоняем Ho об отсутствии взаимосвязи и делаем вывод, что взаимосвязь между переменными статистически значима.

## 4. Выводы
1. В качестве метрики для оценки эффективности работы нового алгоритма для поиска наиболее подходящих анкет использован % мэтчей (число мэтчей / общее число взаимодействий) по 2 группам.  
2. % мэтчей в группе с новым алгоритмом (40,2%) в 2 раза выше % мэтчей в группе со старым алгоритмом (19,5%). 
3. Использование старого/нового алгоритма и мэтч/отказ являются категориальными переменными. Для оценки стат значимости использовался критерий хи-квадрат, который показал стат значимость полученных результатов.
4. Делаем вывод, что новый алгоритм для поиска наиболее подходящих анкет действительно является эффективным и его следует распространить на всех пользователей.


## 5. P.S.
В качестве дополнительной метрики (и для тренировки) можно проанализировать **% успешных юзеров** (у кого случился хоть 1 мэтч): успешные юзеры / всего юзеры.

In [21]:
# Найдем число уникальных юзеров в 2 группах

total_users = df.groupby('group') \
                .agg({'user_id_1' : 'nunique'}) \
                .rename(columns = {'user_id_1' : 'total_users'})
total_users

,total_users
group,
0,501
1,499


In [22]:
# Найдем число успешных юзеров (у кого случился хоть один мэтч) в 2 группах

match_users = df.query('is_match == 1') \
                .groupby('group') \
                .agg({'user_id_1' : 'nunique'}) \
                .rename(columns = {'user_id_1' : 'match_users'})
match_users

,match_users
group,
0,422
1,498


In [23]:
# Объединим таблицы и посчитаем число неуспешных юзеров (у кого не случилось ни одного мэтча)

match_2 = total_users.merge(match_users, on = 'group')
match_2['unmatch_users'] = match_2.total_users - match_2.match_users

In [24]:
# Уберем колонку с общим кол-вом юзеров

match_2 = match_2.drop(columns= 'total_users')
match_2

,match_users,unmatch_users
group,,
0,422,79
1,498,1


У нас 2 категориальные переменные (группа, мэтч) по уникальным пользователям. Поэтому в качестве критерия используем критерий хи-квадрат.  
Зададим гипотезы:  

- $H_0$: взаимосвязи между переменными нет 
- $H_1$: взаимосвязь есть

In [25]:
stat, p, dof, expected = chi2_contingency(match_2)

In [26]:
stat, p

(80.22295132658792, 3.3445968011462423e-19)

In [27]:
if p < 0.05:
    print ('Отклоняем Ho')
else:
    print ('Отклоняем H1')

Отклоняем Ho


p value < 0.05, значим мы отклоняем Ho и считаем, что есть статистически значимая взаимосвязь между показателями.

Считаем % успешных юзеров (успешные юзеры / всего юзеры).

In [28]:
match_2['match'] = match_2.match_users / (match_2.match_users + match_2.unmatch_users) * 100
match_2

,match_users,unmatch_users,match
group,,,
0,422,79,84.231537
1,498,1,99.799599


## Вывод:
% успешных юзеров выше у группы 1, результаты статистически значимы. Делаем вывод, что и данная метрика говорит нам о том, что новый алгоритм для поиска  наиболее подходящих анкет действительно является эффективным и его следует распространить на всех пользователей.